# Module 4: Data viz

## Overview: Geospatial data viz
This notebook will take you through python code very commonly used to import, sort, and visualize some data, in this case a spreadsheet of USGS core data. We will use `pandas` and several packages for geospatial map-making, including `geopandas`, `folium`, and `plotly`.

For questions on this notebook, ask them on the [GEOL 557 slack](https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA)<a href="https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA">
<img src="https://cdn.brandfolder.io/5H442O3W/as/pl546j-7le8zk-ex8w65/Slack_RGB.svg" alt="Go to the GEOl 557 slack" width="100">
</a>

## Instructions
Work through this notebook - there will be several places where you need to fill-in-the-blank or write some code into an open cell. When you are finished, make sure to use the Colab menu (not the browser-level menu) to do the following:
- Expand all the sections - in the Colab menu, choose View --> Expand sections) 
- Save the notebook as a pdf, again using the Colab menu, using File --> Print --> Save as PDF. 

--- 
## Course
**GEOL 557 Earth Resource Data Science I: Fundamentals**. GEOL 557 forms part 2 of the four-part course series for the "Earth Resource Data Science" online graduate certificate at Mines - [learn more about the certificate here](https://online.mines.edu/er/)

Notebook created by **Zane Jobe** and **Thomas Martin**, [CoRE research group](https://core.mines.edu), Colorado School of Mines

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ZaneJobe.svg?style=social&label=Follow%20%40ZaneJobe)](https://twitter.com/ZaneJobe)
and [![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ThomasM_geo.svg?style=social&label=Follow%20%40ThomasM_geo)](https://twitter.com/ThomasM_geo) on Twitter  

-------------------
# OK, let's do some coding!
## First, let's import some packages and install some stuff

In [ ]:
## THIS WILL TAKE ABOUT TWO MINUTES TO RUN...

# install stuff
!pip install geopandas
!pip install earthpy
# !pip install plotly==4.10.0 # needs to be this latest version (Sept 2020), as Colab has an older version as the default
!pip install plotly --upgrade

# import stuff
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import geopandas as gpd
import earthpy as et

import folium

import plotly.express as px

## Import some data
We will use a csv file that has lists all the cores in the USGS Core Research Center in Lakewood, CO. I just downloaded it directly from the website and made no changes to it. That's the whole point of all this, which is to work with data you can easily obtain and not spend time manually changing things in Excel. 

You don't need to do this, but if you want to recreate this dataset, you can download this exact spreadsheet from the [USGS CRC](https://my.usgs.gov/crcwc/). Just click search, don't type anything into the boxes. You should get 16,531 rows (as of Feb 2020), which you can export as a CSV at the bottom of the page. You could do the same things for cuttings by clicking on the cuttings tab (there are 53,456 rows in the cuttings database), but we will just stick with cores for now - 16,000 rows is plenty of data for our first foray into python!

If the code to import the data does not work, you can go to [this csv](https://docs.google.com/spreadsheets/d/1fX8ZyF2Pmx7apcBftvWVVOUlPTCRl014cIASIowAVNE/edit?usp=sharing) in Google Drive - open it in Google Sheets, and then click 'Share' in the upper right hand corner. Copy the link, and then paste it below, replacing the `view?usp=sharing` with `export?format=csv`.

BTW - if instead you go to the file straight in Google Drive, and copy the link from there rather than inside of Google Sheets, you need to do the import a slightly different way (described in this [stackoverflow post](https://stackoverflow.com/questions/56611698/pandas-how-to-read-csv-file-from-google-drive-public)). Why it is different in Sheets vs, Drive, I do not know...

In [ ]:
from google.colab import drive # this mounts Google Drive to this notebook
drive.mount('/content/gdrive')

# these next two things shuoldnt need to be changed if you set up your Google Drive folder correctly (see Module 1)
folder_path = 'gdrive/My Drive/GEOL557_F22/data/' # makes a path
file_name = 'USGS_cores_all.csv' # file name

df=pd.read_csv(folder_path + file_name) # uses pandas to read in the csv as a 'DataFrame' called df

df.head(2) # show the first two lines

## Check out what is in the `DataFrame` we just made

In [ ]:
df.info() # this lists all of the column names and their attributes

In [ ]:
# now let's make a thickness value from the Depth columns, and then add it to the DataFrame

th=df['Max Depth']-df['Min Depth'] # make the variable
df['Thickness']=th                 # assign it to a new column in dataframe called thickness
df.Thickness.describe()            # get some stats

A median of 41 feet per well seems ok, but the mean is 126, so the distribution is extremely log-normal. Notice the max and min are obviously errors in the database. We won't worry about that for now, as we want to check out the bulk statistics of the database. But good to know there may be a few weird values that you would want to filter out. 

Also, note that there are a few thickness values missing, as the dataframe is 16531 rows and the thickness only has 16485 rows (meaning some rows don't have depth values).

In [ ]:
original_length = len(df)

df = df[df.Thickness>0]            # get rid of negative values
df = df[df.Thickness<1000]         # get rid of huge values

dropped = original_length - len(df)

print(str(dropped)+' values dropped out of '+str(original_length)+' values')

## Now let's make some simple plots

In [ ]:
df.plot(kind='scatter', x='Longitude', y='Latitude')

Can you see the Alaska cores?

Also looks like there are some values over in Fiji, but we wont worry about those for now. 

Instead, let's clip Alaska out of the plot and make a bubble plot of core thickness

In [ ]:
plt.scatter(df.Longitude,df.Latitude, s=df.Thickness/50)
plt.xlim([-120, -90])
plt.ylim([30, 50]);

# Can you see the Rockies? This is starting to look like the US, 
# but we can do so much better, as you will see below

### But before we go onto making fancier maps, let's make some bar charts!

In [ ]:
fm_counts=df['Formation'].value_counts() # get value counts of all the formations

sns.barplot(y=fm_counts.index[0:20], 
            x=fm_counts.values[0:20])    # plot it
plt.title('Top 20 Formations with the most cores');

In [ ]:
# Show what types of data are in Colorado
co=df.loc[df['State']=='CO']
co_counts=co['Type'].value_counts()
sns.barplot(y=co_counts.index, x=co_counts.values)
plt.title('Types of core data in Colorado');

## Now you try to make a simple plot
Pick anything! Perhaps a KDE of core thickness from a particular state? Howver, don't fool with lat-long for now, we will spend plenty of time on that below...

In [ ]:
df.State.head()

In [ ]:
# your code here
UT=df.loc[df['State']=='UT']
UT_counts=UT['Age'].value_counts()
sns.barplot(y=UT_counts.index, x=UT_counts.values)
plt.title('Geological ages in Utah');

In [ ]:
df.Field.head()

In [ ]:
WI=df.loc[df['Field']=='WILDCAT']
WI_counts=WI['Type'].value_counts()
sns.barplot(y=WI_counts.index, x=WI_counts.values)
plt.title('Type of core presented  in Wildcat Field');

In [ ]:
df.head()

In [ ]:
df.Thickness.dtype

In [ ]:
plt.figure(figsize=(14,10))
plt.scatter(x='Longitude', y='Latitude', data = df, c='Thickness', vmin=0, vmax=150 )
plt.xlim(-120,-90)
plt.ylim(30,50)
plt.ylabel('Latitude', fontsize=12)
plt.xlabel('Longitude')
plt.title('Cores by lattitude - Color-code Thickness')
plt.colorbar(label='Thickness')


# Basic maps using geopandas

We will use two methods to retrieve data from the internet:
1. geojson files using geopandas
  - `geojson` files are pretty nicely formatted text files. More on that format at [geojson.org](https://geojson.org). An example of creating a geojson file from a pandas DataFrame is shown in a nice notebook [here](https://github.com/gboeing/urban-data-science/blob/master/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb). 

2. zip files using Earthpy and geopandas
  -  Earthpy unzips and stores data on our Colab 'computer'. The one-liners below look complicated, but basically just unzip a file downloaded from the internet and turn it into a GeoDataFrame. For some reason, the zip files won't work in Colab for NaturalEarthData, hence the #1 method above...


In [ ]:
# get basic geopandas data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

'''
get county data for the US from this website
https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

We will do this with Earthpy, which takes the zip file, stores it in Colab, and then unzips it and feeds it to geopandas
'''

counties = gpd.read_file(
    et.data.get_data(
        url=
        'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip'
        )
    )

'''
Now Let's get some custom data from the EIA
I got the data from this website https://www.eia.gov/maps/maps.htm, and here is a summary map: https://www.eia.gov/maps/images/shale_gas_lower48.jpg
'''

plays = gpd.read_file(
    et.data.get_data(
        url=
        'https://www.eia.gov/maps/map_data/TightOil_ShaleGas_Plays_Lower48_EIA.zip'
        )
    )

basins = gpd.read_file(
    et.data.get_data(
        url=
        'https://www.eia.gov/maps/map_data/SedimentaryBasins_US_EIA.zip'
        )
    )

'''
get NaturalEarthdata
the links below copied from https://github.com/nvkelso/natural-earth-vector/tree/master/geojson
but you can also go to this site https://www.naturalearthdata.com/downloads/

For some weird reason, Colab doesnt like Earthpy .zip files for these (see below for an example), so 
we will be using the geojson files directly from Github instead...
'''
coastlines = gpd.read_file('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_50m_coastline.geojson')

countries = gpd.read_file('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_50m_admin_0_boundary_lines_land.geojson')

rivers = gpd.read_file('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_10m_rivers_lake_centerlines.geojson')

states = gpd.read_file('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_1_states_provinces.geojson')

cities = gpd.read_file('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_10m_populated_places.geojson')


Isn't that handy!? You can see where Earthpy put those files, each of which is a zip file consisting of a `.shp` and its brethren. Instead of doing this, you could also:

- Download the files
- click on the little folder icon on the left-hand sidebar
- click the little up-arrow button, and select the zip file and upload them
- then right-click on that file and copy the path and use it to read directly into geopandas using gpd.read_file('filename.shp')

## Let's take a look at some of the data
Each one of these variables is a `GeoDataFrame`, which is basicallt a pandas DataFrame with a `geometry` column. Read more about geometries [in the geopandas docs.](https://geopandas.org/data_structures.html)

In [ ]:
print(type(basins))
basins.head(3) # what does basins look like?

In [ ]:
# Let's make a simple figure

fig, ax = plt.subplots(figsize=[15,15])
world.boundary.plot(ax=ax,color='k') #the boundary method only plots the outlines with no fill
rivers.plot(ax=ax,color='b',linewidth=0.5)
ax.set_aspect('equal')

It would be nice to be able to label these countries or cities. We will use pandas-style subsetting and a `lambda` to do this, this specific lamdba I [found here](https://github.com/shotleft/how-to-python/blob/master/How%20it%20works%20-%20labelling%20districts%20in%20GeoPandas.ipynb)). 

In [ ]:
world.continent.unique() # what continents do we have?

In [ ]:
# Let's subset the data, then plot it

#only south america
south_america = world[world.continent=='South America']
africa = world[world.continent=='Africa']

#plot
fig, axs = plt.subplots(1,2,figsize=[10,5])
south_america.plot(ax=axs[0],color='whitesmoke', edgecolor='k',linewidth=0.5)
africa.plot(ax=axs[1],color='whitesmoke', edgecolor='k',linewidth=0.5)

#labels using a lambda
south_america.apply(lambda x: axs[0].annotate(s=x['name'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);

# instead of the lambda, you can also loop it like this:
for x, y, label in zip(africa.centroid.x, 
                       africa.centroid.y, 
                       africa.name):
    axs[1].text(x, y, label, fontsize = 6, color='k', ha='center')

for ax in axs: ax.set_aspect('equal')


### Now you try
Make a map using the GeoDataFrames that are imported above - see if you can make a map and label things using Geopandas. Maybe one state and it's cities? Or basins and plays? 

In [ ]:
# your code here


In [ ]:

#only US
US = states[states.iso_a2=='US']

#plot
fig, axs = plt.subplots(figsize=[10,15])
US.plot(ax=axs,color='whitesmoke', linewidth=0.35)
plt.xlim(-130,-60)
plt.ylim(20,50)
plt.title('US and States')
#labels using a lambda
US.apply(lambda x: axs.annotate(s=x['name'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);

ax.set_aspect('equal')


### Now let's look at EIA 'oil shale basins':

In [ ]:
fig, ax = plt.subplots(figsize=[15,15])

basins.plot(ax=ax,facecolor='xkcd:blue', alpha=0.2, edgecolor='none') # shade
basins.plot(ax=ax,facecolor='none', edgecolor='xkcd:blue', linewidth=2) # outlines
states.plot(ax=ax,facecolor='none',edgecolor='k',linewidth=1)

ax.scatter(df.Longitude, df.Latitude, s=1, facecolors='none', edgecolors='xkcd:orange')

basins.apply(lambda x: ax.annotate(s=x['NAME'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);

ax.set_xlim([-130,-65])
ax.set_ylim([24,50])
ax.set_aspect('equal');

Looks a bit stretched - this has to do with the coordinate reference system - these are in a geographic coordinate system (WGS84), and we are used to looking at things in Mercator projection are projected in WGS84, but we are used to looking at things in Spherical Mercator, which is a projected coordinate system. Here is a [good intro](https://jcutrer.com/python/learn-geopandas-plotting-usmaps) to this issue. 

Let's take a look at the CRS:

In [ ]:
basins.crs

In [ ]:
# Let's also take our original dataframe and make it a geopandas

# geographic coordinate system
wgs84 = 'EPSG:4326'

df_geo = gpd.GeoDataFrame(df, crs=wgs84, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# projected coordinate systems 
albers_equal_area = 'EPSG:2163' # good for North America, terrible for other places
google_maps = 'EPSG:3857' # a modified mercator projection used in Google Maps

# make into a projected coordinate system
df_geo = df_geo.to_crs(albers_equal_area)
df_geo.crs

In [ ]:
fig, ax = plt.subplots(figsize=[15,15])

basins_proj = basins.to_crs(albers_equal_area)
states_proj = states.to_crs(albers_equal_area)

# only the Lower 48 one-liner
states_proj.drop(states_proj[states_proj.name.isin(['Alaska','Hawaii'])].index, inplace=True) 

basins_proj.plot(ax=ax,facecolor='xkcd:blue green', alpha=0.2, edgecolor='none') # shade
basins_proj.plot(ax=ax,facecolor='none', edgecolor='xkcd:blue green') # outlines
states_proj.plot(ax=ax,facecolor='none',edgecolor='k',linewidth=1)

df_geo.plot(ax=ax,color='xkcd:orange',markersize=1)

basins_proj.apply(lambda x: ax.annotate(s=x['NAME'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);

xlim = ([states_proj.total_bounds[0],  states_proj.total_bounds[2]])
ylim = ([states_proj.total_bounds[1],  states_proj.total_bounds[3]])

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_aspect('equal');

In [ ]:
# compare two projections

fig, axs = plt.subplots(1,2,figsize=[20,10])

# EQUAL AREA
basins_proj.plot(ax=axs[0],facecolor='xkcd:blue green', alpha=0.2, edgecolor='none') # shade
basins_proj.plot(ax=axs[0],facecolor='none', edgecolor='xkcd:blue green') # outlines
states_proj.plot(ax=axs[0],facecolor='none',edgecolor='k',linewidth=1)
df_geo.plot(ax=axs[0],color='xkcd:orange',markersize=1)
basins_proj.apply(lambda x: axs[0].annotate(s=x['NAME'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);
xlim = ([states_proj.total_bounds[0],  states_proj.total_bounds[2]]); ylim = ([states_proj.total_bounds[1],  states_proj.total_bounds[3]])
axs[0].set_xlim(xlim); axs[0].set_ylim(ylim); axs[0].set_aspect('equal');

# GOOGLE MAPS Psuedo Mercator
basins_proj = basins.to_crs(google_maps)
states_proj = states_proj.to_crs(google_maps)
df_geo = df_geo.to_crs(google_maps)

basins_proj.plot(ax=axs[1],facecolor='xkcd:blue green', alpha=0.2, edgecolor='none') # shade
basins_proj.plot(ax=axs[1],facecolor='none', edgecolor='xkcd:blue green') # outlines
states_proj.plot(ax=axs[1],facecolor='none',edgecolor='k',linewidth=1)
df_geo.plot(ax=axs[1],color='xkcd:orange',markersize=1)
basins_proj.apply(lambda x: axs[1].annotate(s=x['NAME'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=8,color='black'),axis=1);
xlim = ([states_proj.total_bounds[0],  states_proj.total_bounds[2]]); ylim = ([states_proj.total_bounds[1],  states_proj.total_bounds[3]])
axs[1].set_xlim(xlim); axs[1].set_ylim(ylim); axs[1].set_aspect('equal');

In [ ]:
# now just the Powder River Basin
cities_sub = cities.to_crs(google_maps)[cities.ADM1NAME=='Wyoming']
df_geo_sub = df_geo[df_geo.State=='WY']

fig, ax = plt.subplots(figsize=[10,10])

basins_proj[basins_proj.NAME=='POWDER RIVER'].plot(ax=ax,facecolor='xkcd:blue green', alpha=0.2, edgecolor='none') # shade
basins_proj[basins_proj.NAME=='POWDER RIVER'].plot(ax=ax,facecolor='none', edgecolor='xkcd:blue green') # outlines
states_proj[states_proj.name=='Wyoming'].plot(ax=ax,facecolor='none',edgecolor='k',linewidth=2)
states_proj[states_proj.name=='Montana'].plot(ax=ax,facecolor='none',edgecolor='k',linewidth=2)

cities_sub.plot(ax=ax,color='k',markersize=20)

df_geo_sub[df_geo_sub.Formation=='PARKMAN'].plot(ax=ax,color='xkcd:orange',markersize=5)

cities_sub.apply(lambda x: ax.annotate(s=x['NAME'], xy=x.geometry.centroid.coords[0],ha='right',fontsize=8,color='black'),axis=1);

ax.set_aspect('equal')
plt.tight_layout();

So, that looks pretty good, but it's pretty basic, and you see that you have to tell geopandas exactly what to do, and you have to label everything manually. There are better ways, which we will see below - but first...

### Your turn!

Pick another basin and formation, and make another map!

In [ ]:
### your code goes here
# figure out a basin and formation using pandas



In [ ]:
# Now plot it using geopandas!



## Folium web-mapping
Folium is a pretty cool package in python that allows you to make interactive, zoomable maps using a Java package called leaflet. We will only scratch the surface of the [capabilities of Folium](https://python-visualization.github.io/folium/quickstart.html) - you can embed these maps on a website, and make all kinds of cool, interactive plots. 

These types of maps are super useful when you need to re-plot the same map over and over, for example, to update a website for weekly rainfall data, or highlight counties that have active wells drilling, etc.

In [ ]:
# define a dataset to map
formation_to_map = df[df.Formation=='NIOBRARA'] # subset the original dataframe

formation_to_map = formation_to_map.dropna(subset=['Latitude']) # get rid of wells with no location information

# now create a map
m = folium.Map(
    location=[42, -105],
    tiles='Stamen Terrain',
    zoom_start=6
)

# and add stuff to that map
for row in formation_to_map.iterrows():
  row_values = row[1]
  location = [row_values['Latitude'], row_values['Longitude']]
  popup = row_values['Well Name']
  folium.Marker(location = location, popup = popup).add_to(m)

m

In [ ]:
# get some info about a particular well

# click on a well and copy the well name, then paste it below 
name = '31-15 MANNING'

df[df['Well Name']==name]

Now let's try another formation, and we will use the OpenStreetMap tiles this time. 

In [ ]:
formation_to_map = df[df.Formation=='PARKMAN'] # subset the original dataframe

formation_to_map = formation_to_map.dropna(subset=['Latitude']) # get rid of wells with no location information

# now create a map
m = folium.Map(
    location=[45, -105],
    tiles='OpenStreetMap',
    zoom_start=6
)

# and add stuff to that map
for row in formation_to_map.iterrows():
  row_values = row[1]
  location = [row_values['Latitude'], row_values['Longitude']]
  popup = row_values['Well Name']
  folium.Marker(location = location, popup = popup).add_to(m)

m

## Plotly

Plotly is pretty sweet - it's even easier than folium, in my opinion. Beware, however, that plotly is a business, and they make money by getting you to sign up for their API and dashboard. So, it's really only partially open-source...

Brendon Hall has a nice plotly notebook that he summarized in [this LinkedIn post](https://www.linkedin.com/pulse/interactive-well-maps-python-brendon-hall). Check that out, as well as the myriad of plotly tutorials online. 

### Let's look at the USGS website
Go check out the [Map based search](https://my.usgs.gov/crcwc/map) at the USGS CRC. Are you ready to recreate that map (and actually make it better) in about ten lines of code? 

### Let's make a quick plotly map

In [ ]:
df_sub = df[df.State=='WY']
df_sub = df_sub[['Well Name','Latitude','Longitude','Formation','Thickness']]
df_sub = df_sub.dropna()

fig = px.scatter_mapbox(df_sub, lat="Latitude", lon="Longitude",
                        color='Formation', 
                        zoom=5, height=600,
                        hover_data={'Well Name': True,
                                    'Latitude': False,
                                    'Longitude': False,
                                    'Formation': True,
                                    'Thickness': True}
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":200,"t":20,"l":200,"b":0})
fig.show()

![Damn](https://media1.tenor.com/images/3e64ea8c3ee1147ec50376620f984792/tenor.gif?itemid=5580082)

In [ ]:
# Now you try a plotly map! 
df_sub = df[df.State=='CO']
df_sub = df_sub[['Well Name','Latitude','Longitude','Formation','Thickness']]
df_sub = df_sub.dropna()

fig = px.scatter_mapbox(df_sub, lat="Latitude", lon="Longitude",
                        color='Formation', 
                        zoom=5, height=600,
                        hover_data={'Well Name': True,
                                    'Latitude': False,
                                    'Longitude': False,
                                    'Formation': True,
                                    'Thickness': True}
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":200,"t":20,"l":200,"b":0})
fig.show()
# your code here

So, see, being a fancy data scientist isn't rocket science, it's mostly just googling things. 
![Googling is half](https://img.ifunny.co/images/072cea68327e54d5816c1a2f921f2958440586b2da0af7e9656f5255d2c7ab7b_1.jpg)

---------

# The End
![Work here is done](https://media1.tenor.com/images/168afe17abdf88b0d439c901f134a6f4/tenor.gif?itemid=4705793)